In [1]:
from transformers import pipeline

c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prabhav.gupta\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limi

In [3]:
result = classifier('I love using Hugging Face!')

In [4]:
print(result)

[{'label': 'POSITIVE', 'score': 0.9997085928916931}]


In [7]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
import numpy as np

In [8]:
# Load pre-trained DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prabhav.gupta\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\prabhav.gupta\OneDrive - 

In [9]:
def extract_features(logs):
    inputs = tokenizer(logs, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use [CLS] token as feature vector

In [11]:
import csv
import os

In [22]:
def load_logs_from_sources():
    input_csv_name = "GPN_Syslog_10000.csv"
    input_dir = "data"
    input_csv_path = os.path.join(input_dir,input_csv_name)
    logs = []
    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
        # Create CSV reader and writer
        csv_reader = csv.reader(infile)
        next(csv_reader) # skipping the header row
        # Process each row in the input CSV
        for row in csv_reader:
            if len(row) < 3:
                continue
            logs.append(row[2])
    
    return logs        
            

In [23]:
# Load and preprocess your logs
logs = load_logs_from_sources()  # Implement this function to load your log data


In [25]:
print(logs[2])

Jun 11 2024 11:52:51 GPN-0277_DAV-CLI_AR12-01 %%01SNMP/5/SNMP_LOG_IP_UNLOCK(l)[120208]:The IP address of request message was unlocked because timeout. (SourceIPType=1, SourceIP=172.31.251.25)


In [26]:
inputs = tokenizer(logs, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [28]:
print(inputs)

{'input_ids': tensor([[  101, 12022,  2340,  ...,     0,     0,     0],
        [  101, 12022,  2340,  ...,     0,     0,     0],
        [  101, 12022,  2340,  ...,     0,     0,     0],
        ...,
        [  101, 12022,  2340,  ...,     0,     0,     0],
        [  101, 12022,  2340,  ...,     0,     0,     0],
        [  101, 12022,  2340,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [29]:
# Extract features
features = extract_features(logs)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 15728640000 bytes.

In [1]:
import json

def standardising_live_message(message):
    message_dict = json.loads(message)
    raw_msg = message_dict.get('payload','')
    if raw_msg != '':
        index = raw_msg.find('>')
        msg_str = raw_msg[index+1:]
        return msg_str
    else:
        return ''

In [9]:
a = standardising_live_message('{"schema":null,"payload":"<188>Sep 24 2024 10:01:38 GPN-0231_BOI-EDU_AR12-01 LINE/4/USERLOGINFAIL:OID 1.3.6.1.4.1.2011.5.25.207.2.3 A user login fail. (UserIndex=0, UserName=Console, UserIP=Console0, UserChannel=CON0) "}')

In [10]:
a

'Sep 24 2024 10:01:38 GPN-0231_BOI-EDU_AR12-01 LINE/4/USERLOGINFAIL:OID 1.3.6.1.4.1.2011.5.25.207.2.3 A user login fail. (UserIndex=0, UserName=Console, UserIP=Console0, UserChannel=CON0) '

In [13]:
input_csv = 'batch_data/batch_1.csv'
import csv

In [16]:
with open(input_csv, mode='r', newline='', encoding='utf-8') as infile:
    # Create CSV reader
    csv_reader = csv.reader(infile)
    next(csv_reader) # skipping the header row
    for row in csv_reader:
        print(row)
        print(standardising_live_message(row[0]))
                

['{"schema":null,"payload":"<188>Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92240]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(53077)-> 8.8.8.8(53) (2) packets. "}']
Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92240]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(53077)-> 8.8.8.8(53) (2) packets. 
['{"schema":null,"payload":"<188>Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92241]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(51499)-> 8.8.4.4(53) (1) packets. "}']
Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92241]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(51499)-> 8.8.4.4(53) (1) packets. 
['{"schema":null,"payload":"<188>Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92245]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(34849)-> 8.8.8.8(53) (2) packets. "}']
Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92245]:Ipv4 acl 3

In [3]:
import os

batch_data_dir = "batch_data" # raw data
processed_data_dir = "processed_data" # final processed data
batch_number = 1
batch_fill = 0
batch_size = 10000
batch_filename = f"output_batch_{batch_number}.csv"
output_filepath = os.path.join(processed_data_dir, batch_filename)
# Define the directory where the CSV files are stored
current_directory = os.getcwd()
print(current_directory)
directory_path = os.path.join(current_directory,batch_data_dir)

c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert


In [4]:
import os
import time
import pandas as pd
import csv
from syslog_processing import Processing


object_processing = Processing()
if not os.path.isdir(processed_data_dir):
        os.makedirs(processed_data_dir)
        
with open(output_filepath, mode= 'w', newline='', encoding='utf-8') as outfile:
    csv_writer = csv.DictWriter(outfile, fieldnames=object_processing.headers)

    # Write the header to the new CSV file
    csv_writer.writeheader()

In [5]:
import os
# Iterate over files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):  # Check if the file is a CSV
        
        input_filepath = os.path.join(directory_path, filename)
        print(f"-----------{input_filepath}")

        with open(input_filepath, mode='r', newline='', encoding='utf-8') as infile, \
            open(output_filepath, mode= 'a', newline='', encoding='utf-8') as outfile:
                
            # Create CSV reader
            csv_reader = csv.reader(infile)
            next(csv_reader) # skipping the header row
            csv_writer = csv.writer(outfile)
            
            for row in csv_reader:
                print(row[0])
                print(type(row[0]))
                message_dict = object_processing.end_to_end_processing_single_live_message(json_string_live_message=row[0])
                print("-------")
                print(message_dict) 
                data_to_write = {
                    'timestamp_of_message': message_dict.get('timestamp_of_message', ''),
                    'year': message_dict.get('year', ''),
                    'month': message_dict.get('month', ''),
                    'day_of_month': message_dict.get('day_of_month', ''),
                    'hour': message_dict.get('hour', ''),
                    'minute': message_dict.get('minute', ''),
                    'second': message_dict.get('second', ''),
                    'millisecond': message_dict.get('millisecond', ''),
                    'day_of_week': message_dict.get('day_of_week', ''),
                    'day_of_year': message_dict.get('day_of_year', ''),
                    'week_of_year': message_dict.get('week_of_year', ''),
                    'is_weekend': message_dict.get('is_weekend', ''),
                    'seconds_since_midnight': message_dict.get('seconds_since_midnight', ''),
                    'time_of_day_bucket': message_dict.get('time_of_day_bucket', ''),
                    'device_identifier': message_dict.get('device_identifier', ''),
                    'vendor': message_dict.get('vendor', ''),
                    'physical_site_id': message_dict.get('physical_site_id', ''),
                    'geolocation_code': message_dict.get('geolocation_code', ''),
                    'device_role': message_dict.get('device_role', ''),
                    'device_function': message_dict.get('device_function', ''),
                    'device_model_number': message_dict.get('device_model_number', ''),
                    'device_importance': message_dict.get('device_importance', ''),
                    'version_number': message_dict.get('version_number', ''),
                    'module_name': message_dict.get('module_name', ''),
                    'severity': message_dict.get('severity', ''),
                    'log_type': message_dict.get('log_type', ''),
                    'log_status': message_dict.get('log_status', ''),
                    'cid': message_dict.get('cid', ''),
                    'oid': message_dict.get('oid', ''),
                    'sequence_number': message_dict.get('sequence_number', ''),
                    'description': message_dict.get('description', ''),
                    'device_with_description': message_dict.get('device_with_description', ''),
                    'metadata': message_dict.get('metadata', ''),
                    'is_unhappy': message_dict.get('is_unhappy', '')
                }
                csv_writer.writerow(data_to_write)
                print("here")

-----------c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\batch_data\batch_1.csv
{"schema":null,"payload":"<188>Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92240]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(53077)-> 8.8.8.8(53) (2) packets. "}
<class 'str'>
-------
{'timestamp_of_message': 'Sep 24 2024 11:47:10', 'year': 2024, 'month': 9, 'day_of_month': 24, 'hour': 11, 'minute': 47, 'second': 10, 'millisecond': 0, 'day_of_week': 1, 'day_of_year': 268, 'week_of_year': 39, 'is_weekend': 0, 'seconds_since_midnight': 42430, 'time_of_day_bucket': 'Morning', 'device_identifier': 'GPN-1220-PEL-PRI-SCH-AR12-01', 'vendor': 'GPN', 'physical_site_id': '1220', 'geolocation_code': 'PEL', 'device_role': 'PRI', 'device_function': 'SCH', 'device_model_number': 'AR12', 'device_importance': '01', 'version_number': '01', 'module_name': 'QOS', 'severity': '4', 'log_type': 'SACL_LOG', 'log_status': '(l)', 'cid': '', 'description

In [24]:
import os
import time
import pandas as pd
import csv
from syslog_processing import Processing

batch_data_dir = "batch_data" # raw data
processed_data_dir = "processed_data2" # final processed data
object_processing = Processing()
if not os.path.isdir(processed_data_dir):
    os.makedirs(processed_data_dir)

In [25]:
batch_number = 1
batch_fill = 0
batch_size = 10000
batch_filename = f"output_batch_{batch_number}.csv"
output_filepath = os.path.join(processed_data_dir, batch_filename)
# Define the directory where the CSV files are stored
current_directory = os.getcwd()
print(current_directory)
directory_path = os.path.join(current_directory,batch_data_dir)

c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert


In [26]:
with open(output_filepath, mode= 'w', newline='', encoding='utf-8') as outfile:
    csv_writer = csv.DictWriter(outfile, fieldnames=object_processing.headers)

    # Write the header to the new CSV file
    csv_writer.writeheader()


In [27]:
input_filepath = os.path.join(directory_path, 'batch_1.csv')
print(f"-----------{input_filepath}")

with open(input_filepath, mode='r', newline='', encoding='utf-8') as infile, \
    open(output_filepath, mode= 'a', newline='', encoding='utf-8') as outfile:
        
    # Create CSV reader
    csv_reader = csv.reader(infile)
    next(csv_reader) # skipping the header row
    csv_writer = csv.DictWriter(outfile, fieldnames=object_processing.headers)
    # csv_writer = csv.writer(outfile)
    
    for row in csv_reader:
        print(row[0])
        print(type(row[0]))
        message_dict = object_processing.end_to_end_processing_single_live_message(json_string_live_message=row[0])
        print("-------")
        print(message_dict)
        
        # Write the extracted data to the output CSV file
            
        data_to_write = {
            'timestamp_of_message': message_dict.get('timestamp_of_message', ''),
            'year': message_dict.get('year', ''),
            'month': message_dict.get('month', ''),
            'day_of_month': message_dict.get('day_of_month', ''),
            'hour': message_dict.get('hour', ''),
            'minute': message_dict.get('minute', ''),
            'second': message_dict.get('second', ''),
            'millisecond': message_dict.get('millisecond', ''),
            'day_of_week': message_dict.get('day_of_week', ''),
            'day_of_year': message_dict.get('day_of_year', ''),
            'week_of_year': message_dict.get('week_of_year', ''),
            'is_weekend': message_dict.get('is_weekend', ''),
            'seconds_since_midnight': message_dict.get('seconds_since_midnight', ''),
            'time_of_day_bucket': message_dict.get('time_of_day_bucket', ''),
            'device_identifier': message_dict.get('device_identifier', ''),
            'vendor': message_dict.get('vendor', ''),
            'physical_site_id': message_dict.get('physical_site_id', ''),
            'geolocation_code': message_dict.get('geolocation_code', ''),
            'device_role': message_dict.get('device_role', ''),
            'device_function': message_dict.get('device_function', ''),
            'device_model_number': message_dict.get('device_model_number', ''),
            'device_importance': message_dict.get('device_importance', ''),
            'version_number': message_dict.get('version_number', ''),
            'module_name': message_dict.get('module_name', ''),
            'severity': message_dict.get('severity', ''),
            'log_type': message_dict.get('log_type', ''),
            'log_status': message_dict.get('log_status', ''),
            'cid': message_dict.get('cid', ''),
            'oid': message_dict.get('oid', ''),
            'sequence_number': message_dict.get('sequence_number', ''),
            'description': message_dict.get('description', ''),
            'device_with_description': message_dict.get('device_with_description', ''),
            'metadata': message_dict.get('metadata', ''),
            'is_unhappy': message_dict.get('is_unhappy', '')
        }
        csv_writer.writerow(data_to_write)

-----------c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\batch_data\batch_1.csv
{"schema":null,"payload":"<188>Sep 24 2024 11:47:10 GPN-1220_PEL-PRI-SCH_AR12-01 %%01QOS/4/SACL_LOG(l)[92240]:Ipv4 acl 3500,rule 120 deny 17 10.197.119.254(53077)-> 8.8.8.8(53) (2) packets. "}
<class 'str'>
-------
{'timestamp_of_message': 'Sep 24 2024 11:47:10', 'year': 2024, 'month': 9, 'day_of_month': 24, 'hour': 11, 'minute': 47, 'second': 10, 'millisecond': 0, 'day_of_week': 1, 'day_of_year': 268, 'week_of_year': 39, 'is_weekend': 0, 'seconds_since_midnight': 42430, 'time_of_day_bucket': 'Morning', 'device_identifier': 'GPN-1220-PEL-PRI-SCH-AR12-01', 'vendor': 'GPN', 'physical_site_id': '1220', 'geolocation_code': 'PEL', 'device_role': 'PRI', 'device_function': 'SCH', 'device_model_number': 'AR12', 'device_importance': '01', 'version_number': '01', 'module_name': 'QOS', 'severity': '4', 'log_type': 'SACL_LOG', 'log_status': '(l)', 'cid': '', 'description

In [28]:
'0001' == '1'

False

In [36]:
from syslog_processing import Processing

In [51]:
def get_specific_device_with_descriptions_formatted(input_csv_path, physical_site_id, geolocation_code, device_role, device_model_number, device_importance):
        '''
        only use if the input csv is already in processed format:
        
        timestamp_of_message	year	month	day_of_month	hour	minute	second	millisecond	day_of_week	day_of_year	week_of_year	is_weekend	seconds_since_midnight	time_of_day_bucket	device_identifier	vendor	physical_site_id	geolocation_code	device_role	device_function	device_model_number	device_importance	version_number	module_name	severity	log_type	log_status	cid	oid	sequence_number	description	device_with_description	metadata	is_unhappy
Sep 24 2024 10:02:04	2024	9	24	10	2	4	0	1	268	39	0	36124	Morning	GPN-0001-TLK-CR--M14-01	GPN	1	TLK	CR		M14	1	1	SSH	5	SSH_USER_LOGIN	(s)	0x80930446			The SSH user succeeded in logging in. 	GPN-0001-TLK-CR--M14-01 -> The SSH user succeeded in logging in. 	ServiceType=snetconf, UserName=nms_admin, UserAddress=172.31.25.125, LocalAddress=172.31.0.3, VPNInstanceName=_public_	

        '''
        filtered_desc = []
        with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
            # Create CSV reader
            csv_reader = csv.reader(infile)
            next(csv_reader) # skipping the header row
            
            # Process each row in the input CSV
            for row in csv_reader:
                # print(row)
                # Extract the relevant fields
                extracted_data = {}
                extracted_data['physical_site_id'] = row[16]
                # print(row[16])
                extracted_data['geolocation_code'] = row[17]
                # print(row[17])
                extracted_data['device_role'] = row[18]
                # print(row[18])
                extracted_data['device_model_number'] = row[20]
                # print(row[20])
                extracted_data['device_importance'] = row[21]
                # print(row[21])
                extracted_data['device_with_description'] = row[31]
                if extracted_data['physical_site_id'] == physical_site_id and extracted_data['geolocation_code'] == geolocation_code and extracted_data['device_role'] == device_role and extracted_data['device_model_number'] == device_model_number:  
                    # storing all specific descriptions in a variable
                    print('here')
                    print(extracted_data['device_with_description'])
                    formatted_disc = extracted_data.get('device_with_description', '').lower().strip()
                    filtered_desc.append(formatted_disc)   
        
        return filtered_desc


In [52]:
input_csv_path = 'processed_data/output_batch_1.csv'

In [53]:
get_specific_device_with_descriptions_formatted(input_csv_path= input_csv_path,physical_site_id='0001', geolocation_code='TLK', device_role='CR', device_model_number='M14', device_importance='01')

here
GPN-0001-TLK-CR--M14-01 -> The SSH user succeeded in logging in. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GPN-0001-TLK-CR--M14-01 -> The SSH user succeeded in logging in. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GBN-0001-TLK-CR--M14-02 -> The SSH user failed to login. 
here
GBN-0001-TLK-CR--M14-02 -> IP was locked. 


['gpn-0001-tlk-cr--m14-01 -> the ssh user succeeded in logging in.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gpn-0001-tlk-cr--m14-01 -> the ssh user succeeded in logging in.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0001-tlk-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0001-tlk-cr--m14-02 -> ip was locked.']